# **Twitter Sentiment Classification Project**

- [1. Introduction](#1-introduction)
  - [Recurrent Neural Networks](#recurrent-neural-networks)
  - [High Level Steps](#high-level-steps)

---

## **1. Introduction**

In this project, we tackle the **binary classification** of **Twitter messages** into **positive** and **negative** categories. We are provided with two separate directories containing labeled tweets:

- Directory **0**: Negative tweets
- Directory **1**: Positive tweets

Our approach focuses on a detailed exploration of a single model family rather than broadly comparing multiple families, which could lead to overly generic solutions. By conducting a series of experiments with an emphasis on originality, we aim to dig deeper into the specific characteristics of the chosen model family. This approach allows us to experiment with architecture, hyperparameters, and training strategies in a more targeted way, ultimately evaluating the accuracy of the models on the validation data properly extracted from the dataset.

### **Recurrent Neural Networks**

We chose to focus on experimenting with Recurrent Neural Networks (RNNs) for the sentiment analysis task. RNNs are particularly well-suited for processing sequential data, as they maintain a hidden state that captures the influence of previous input sequences. Tweets, being short and concise, often require understanding the flow of sentiment within a limited context. RNNs excel at this by processing one word at a time, retaining contextual information from previous words to identify patterns that signal positive or negative sentiment.

Although Transformers are the state-of-the-art for many NLP tasks, we specifically opted not to use them here. Transformers are optimized for modeling long-range dependencies and contextual relationships across entire documents. While highly effective, their self-attention mechanism can be unnecessary for short texts like tweets. Additionally, Transformers are computationally intensive, demanding significant resources for training and inference. For a simpler binary classification task, this added complexity can result in inefficiencies without a proportional gain in performance.

By choosing RNNs, we focus on a model that is both resource-efficient and well-matched to the specific demands of analyzing short texts, ensuring effective results without unnecessary computational overhead.

### **High Level Steps**

From a high level, the project will follow these steps:

1. Load and preprocess the text data.
2. Experiment with **different model configurations**.
3. Evaluate and compare these configurations using validation data and standard metrics.
4. Test the final model(s) on **personal tweets** to see how they generalize.

---


In [27]:
%%capture
import os
import logging
import warnings
import math
import numpy as np
import tensorflow as tf
import pandas as pd
from rich.table import Table
from rich.console import Console
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing import text_dataset_from_directory
from sklearn.metrics import confusion_matrix, classification_report

# Ignore warnings
warnings.filterwarnings("ignore")
# Removes info and warning messages from Tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel(logging.ERROR)

os.environ['PYTHONWARNINGS'] = 'ignore'

logging.getLogger('tensorflow').setLevel(logging.ERROR)

console = Console()

def create_rich_table(data, headers, title="Table"):
    table = Table(title=title, show_lines=True)

    # Add headers to the table
    for header in headers:
        table.add_column(header, justify="center")

    # Add rows to the table
    for row in data:
        table.add_row(*map(str, row))

    return table


## **2. Dataset Loading and Preprocessing**

In this section, we focus on loading the dataset, preprocessing the text, and applying vectorization. Our goal is to maintain a modular approach throughout the project to ensure reusability and adaptability of functions, particularly for experimenting with different hyperparameter configurations (e.g., `max_tokens`, `output_sequence_length`).

**Steps Involved**

1. Dataset Loading
2. Creating a `TextVectorization` layer, adaptable to various text preprocessing configurations

The dataset is loaded through the following configurations:

- Batch Size: 32. We opted for a memory-efficient approach by using smaller batch sizes. Since RNNs process data sequentially, larger batches would have significantly increased memory usage. Additionally, given the not too large size of the dataset, smaller batch sizes enable more frequent gradient updates per epoch.

- Splits: 70% for training, 20% for validation and 10% for testing

For the `TextVectorization` layer we used a custom standardization function, tailored for twitter messages, specific for sentyment analysis. In particular, the following rules are applied:

1. Convert to Lowercase
2. Remove HTTP URLs
3. Remove Hashtags
4. Removed hashtags
5. Keep only alphanumeric, spaces, and specific punctuation (!, ?, ...), useful for sentyment analysis tasks.


In [28]:
DATASET_DIR = "../TwitterParsed"


def load_dataset(
    data_dir, batch_size=32, validation_split=0.2, test_split=0.1, seed=42
):

    # Training dataset
    train_ds = tf.keras.utils.text_dataset_from_directory(
        data_dir,
        batch_size=batch_size,
        validation_split=validation_split + test_split,  # Total non-training data
        subset="training",
        seed=seed,
    )

    # Split containing validation and test sets
    val_and_test_ds = tf.keras.utils.text_dataset_from_directory(
        data_dir,
        batch_size=batch_size,
        validation_split=validation_split + test_split,
        subset="validation",
        seed=seed,
    )

    # Further split validation into validation and test datasets
    val_size = math.floor(
        (validation_split / (validation_split + test_split)) * len(val_and_test_ds)
    )

    val_ds = val_and_test_ds.take(val_size)
    test_ds = val_and_test_ds.skip(val_size)

    return train_ds, val_ds, test_ds


def create_vectorization_layer(max_tokens=10000, sequence_length=100):

    # Custom standardization function tailored for tweets
    def custom_standardize(input_text: tf.Tensor) -> tf.Tensor:
        # Lowercase the text
        lowercase_text = tf.strings.lower(input_text)
        # Remove URLs
        text_without_urls = tf.strings.regex_replace(lowercase_text, r"http\S+", " ")
        # Remove mentions (e.g., @username)
        text_without_mentions = tf.strings.regex_replace(
            text_without_urls, r"@\w+", " "
        )
        # Replace hashtags with just the word (e.g., #happy -> happy)
        text_without_hashtags = tf.strings.regex_replace(
            text_without_mentions, r"#", ""
        )
        # Remove single periods (.) but retain the multiple ones
        text_without_single_dots = tf.strings.regex_replace(
            text_without_hashtags, r"(?<!\.)\.(?!\.)", ""
        )
        # Collapse multiple dots (.....) into a single ellipsis (...) for consistency
        text_with_ellipses = tf.strings.regex_replace(
            text_without_single_dots, r"\.{2,}", "..."
        )
        # Keep only alphanumeric, spaces, and specific punctuation (!, ?, ...)
        cleaned_text = tf.strings.regex_replace(
            text_with_ellipses, r"[^a-z0-9\s!?...]", ""
        )
        return cleaned_text

    # Create the TextVectorization layer
    vectorizer = TextVectorization(
        max_tokens=max_tokens,  # Vocabulary size
        output_mode="int",  # Map tokens to integers
        output_sequence_length=sequence_length,  # Pad/Truncate to sequence length
        standardize=custom_standardize,  # Use the custom standardization logic
    )

    return vectorizer


# Count dataset samples
def count_samples(dataset):
    return sum(1 for _ in dataset.unbatch())


def compute_class_distribution(dataset):
    neg_count = 0
    pos_count = 0
    for _, label in dataset.unbatch():
        if label.numpy() == 0:
            neg_count += 1
        else:
            pos_count += 1
    total = neg_count + pos_count
    neg_percent = (neg_count / total) * 100 if total > 0 else 0
    pos_percent = (pos_count / total) * 100 if total > 0 else 0
    return neg_count, pos_count, neg_percent, pos_percent


def create_dataset_summary_table(train_ds, val_ds, test_ds):
    # Calculate counts and class distributions
    train_count = count_samples(train_ds)
    val_count = count_samples(val_ds)
    test_count = count_samples(test_ds)
    total_count = train_count + val_count + test_count

    train_neg, train_pos, train_neg_percent, train_pos_percent = (
        compute_class_distribution(train_ds)
    )
    val_neg, val_pos, val_neg_percent, val_pos_percent = compute_class_distribution(
        val_ds
    )
    test_neg, test_pos, test_neg_percent, test_pos_percent = compute_class_distribution(
        test_ds
    )

    # Prepare the data for the table
    data = [
        [
            "Training",
            train_count,
            f"{train_neg_percent:.2f}%",
            f"{train_pos_percent:.2f}%",
        ],
        ["Validation", val_count, f"{val_neg_percent:.2f}%", f"{val_pos_percent:.2f}%"],
        ["Testing", test_count, f"{test_neg_percent:.2f}%", f"{test_pos_percent:.2f}%"],
        [
            "Total",
            total_count,
            f"{((train_neg + val_neg + test_neg) / total_count) * 100:.2f}%",
            f"{((train_pos + val_pos + test_pos) / total_count) * 100:.2f}%",
        ],
    ]

    # Headers for the table
    headers = ["Dataset", "Number of Tweets", "Negative %", "Positive %"]

    # Creeate a rich table
    rich_table = create_rich_table(data, headers, title="Dataset Summary")

    return rich_table


# Load dataset in training (70%), validation (20%), and test (10%) sets
train_ds, val_ds, test_ds = load_dataset(DATASET_DIR)
summary_table = create_dataset_summary_table(train_ds, val_ds, test_ds)
console.print(summary_table)

Found 149985 files belonging to 2 classes.
Using 104990 files for training.
Found 149985 files belonging to 2 classes.
Using 44995 files for validation.


                      Dataset Summary                      
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃  Dataset   ┃ Number of Tweets ┃ Negative % ┃ Positive % ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│  Training  │      104990      │   50.08%   │   49.92%   │
├────────────┼──────────────────┼────────────┼────────────┤
│ Validation │      30016       │   49.92%   │   50.08%   │
├────────────┼──────────────────┼────────────┼────────────┤
│  Testing   │      14979       │   49.90%   │   50.10%   │
├────────────┼──────────────────┼────────────┼────────────┤
│   Total    │      149985      │   50.03%   │   49.97%   │
└────────────┴──────────────────┴────────────┴────────────┘